In [46]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [47]:


# Define the function to create and train the LSTM model
def create_model_and_train(df):
    df.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    df['log_returns'] = np.log(df['Close']).diff().dropna()

    # Create a scaler to normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

    # Split the data into training and test sets
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    # Prepare the training data for LSTM
    X_train_lstm, y_train_lstm = [], []
    lookback_lstm = 10  # Adjust the lookback window for LSTM

    for i in range(lookback_lstm, len(train_data)):
        X_train_lstm.append(train_data[i-lookback_lstm:i, 0])
        y_train_lstm.append(train_data[i, 0])

    X_train_lstm, y_train_lstm = np.array(X_train_lstm), np.array(y_train_lstm)

    # Reshape the input data for LSTM
    X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], 1))

    # Create and fit the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(units=1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error' , run_eagerly=True)
    lstm_model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32)

    return lstm_model, scaler, lookback_lstm

# Define the function to make predictions using the trained model


In [48]:
def make_predictions(df, lstm_model, scaler, lookback_lstm):
    # Prepare the test data for LSTM
    inputs_lstm = df['Close'].values[len(df) - lookback_lstm:]
    inputs_lstm = inputs_lstm.reshape(-1, 1)
    inputs_lstm = scaler.transform(inputs_lstm)

    X_test_lstm = []

    for i in range(lookback_lstm, len(inputs_lstm)):
        X_test_lstm.append(inputs_lstm[i-lookback_lstm:i, 0])

    X_test_lstm = np.array(X_test_lstm)

    # Reshape the input data for LSTM
    X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], lookback_lstm, 1))

    # Make predictions using the LSTM model
    predicted_prices_lstm = lstm_model.predict(X_test_lstm)
    predicted_prices_lstm = scaler.inverse_transform(predicted_prices_lstm)

    # Return the predicted prices
    return predicted_prices_lstm


# Example usage
# Assuming you have a new dataset stored in a DataFrame named 'new_data'


In [49]:
train_data = pd.read_csv('STOCK_INDEX.csv')

# Create and train the LSTM model
model, scaler, lookback = create_model_and_train(train_data)

new_data2=pd.read_csv('sample_input.csv')
# Make predictions using the trained model
predictions = make_predictions(new_data2, model, scaler, lookback)

# Print the predicted prices
print(predictions)


Epoch 1/10
68/68 [==============================] - 9s 132ms/step - loss: 0.0060
Epoch 2/10
68/68 [==============================] - 10s 144ms/step - loss: 2.0391e-04
Epoch 3/10
68/68 [==============================] - 8s 124ms/step - loss: 1.9938e-04
Epoch 4/10
68/68 [==============================] - 9s 137ms/step - loss: 2.0581e-04
Epoch 5/10
68/68 [==============================] - 9s 136ms/step - loss: 2.1158e-04
Epoch 6/10
68/68 [==============================] - 9s 137ms/step - loss: 1.9004e-04
Epoch 7/10
68/68 [==============================] - 9s 135ms/step - loss: 1.8628e-04
Epoch 8/10
68/68 [==============================] - 9s 131ms/step - loss: 1.9026e-04
Epoch 9/10
68/68 [==============================] - 10s 152ms/step - loss: 1.8913e-04
Epoch 10/10
68/68 [==============================] - 9s 136ms/step - loss: 1.8705e-04


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.